In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from sdgym import benchmark
from sdgym import load_dataset
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from synthsonic.models.kde_copula_nn_pdf import KDECopulaNNPdf
from synthsonic.models.categorical_utils import categorical_round, vec_translate, categorical_frequency_mapping, \
            categorical_frequency_inverse_mapping, encode_one_hot, decode_one_hot

In [3]:
def KDECopulaNNPdf_woKDE_OneHotEncoded(real_data, categorical_columns, ordinal_columns):
    
    all_features = list(range(real_data.shape[1]))
    numerical_features = list(set(all_features) - set(categorical_columns+ordinal_columns))
    
    ## One hot encode the categorical features
    unique_values, ohe = encode_one_hot(real_data, categorical_columns)
    categorical_np = np.array(ohe)
    
    n_samples = real_data.shape[0]
    n_features = real_data.shape[1]
    
    ## Append the categorical one hot encoded data to numerical and ordinal
    data = np.float64(np.hstack((real_data[:, numerical_features+ordinal_columns], categorical_np)))

    kde = KDECopulaNNPdf(use_KDE=False, clf=MLPClassifier(random_state=0, max_iter=500, early_stopping=True))
    kde = kde.fit(data)
    X_gen, sample_weight = kde.sample(n_samples)
    X_gen = np.float32(X_gen)
    
    
    X_final = decode_one_hot(X_gen, categorical_columns, unique_values, n_features)
    X_final[:, numerical_features+ordinal_columns] = X_gen[:, numerical_features+ordinal_columns]
    print(X_final.shape)
    return X_final
    

In [4]:
scores = benchmark(synthesizers=[KDECopulaNNPdf_woKDE_OneHotEncoded],  datasets=['adult'])
scores

Transforming variables.
Fitting and calibrating classifier.
Model = rho: 0.5, number of selected non-linear variables: 91
(22561, 15)
Transforming variables.
Fitting and calibrating classifier.
Model = rho: 0.5, number of selected non-linear variables: 91
(22561, 15)
Transforming variables.
Fitting and calibrating classifier.
Model = rho: 0.5, number of selected non-linear variables: 91
(22561, 15)


,adult/accuracy,adult/f1,timestamp
CLBNSynthesizer,0.760117,0.291326,2020-04-12 09:41:35.096775
CTGANSynthesizer,0.784217,0.597420,2020-04-12 09:41:35.096775
IdentitySynthesizer,0.825608,0.661600,2020-04-12 09:41:35.096775
IndependentSynthesizer,0.654425,0.174515,2020-04-12 09:41:35.096775
MedganSynthesizer,0.593683,0.276167,2020-04-12 09:41:35.096775
TableganSynthesizer,0.798850,0.426258,2020-04-12 09:41:35.096775
TVAESynthesizer,0.810817,0.630175,2020-04-12 09:41:35.096775
UniformSynthesizer,0.500242,0.301878,2020-04-12 09:41:35.096775
VEEGANSynthesizer,0.718792,0.162142,2020-04-12 09:41:35.096775
KDECopulaNNPdf_woKDE_OneHotEncoded,0.501425,0.314894,2020-09-23 14:44:21.423168


In [5]:
def KDECopulaNNPdf_woKDE_FreqMapping(real_data, categorical_columns, ordinal_columns):
    
    all_features = list(range(real_data.shape[1]))
    numerical_features = list(set(all_features) - set(categorical_columns+ordinal_columns))
    data = np.float64(real_data)
    
    n_samples = data.shape[0]
    n_features = data.shape[1]
    
    data, inv_mappings = categorical_frequency_mapping(data, categorical_columns)

    kde = KDECopulaNNPdf(use_KDE=False, clf=MLPClassifier(random_state=0, max_iter=500, early_stopping=True))
    kde = kde.fit(data)
    X_gen, sample_weight = kde.sample(n_samples)
    
    X_gen[:, categorical_columns] = np.round(X_gen[:, categorical_columns])
    X_final = categorical_frequency_inverse_mapping(X_gen, categorical_columns, inv_mappings)
    return X_final

In [6]:
df, categorical_columns, ordinal_columns = load_dataset('adult')
X_gen = KDECopulaNNPdf_woKDE_FreqMapping(df, categorical_columns, ordinal_columns)

Transforming variables.
Fitting and calibrating classifier.
Model = rho: 0.5, number of selected non-linear variables: 14


In [7]:
scores = benchmark(synthesizers=[KDECopulaNNPdf_woKDE_FreqMapping],  datasets=['adult'])
scores

Transforming variables.
Fitting and calibrating classifier.
Model = rho: 0.5, number of selected non-linear variables: 14


/Users/lorrainedalmeida/Work/Experiment_Week/synthsonic/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


Transforming variables.
Fitting and calibrating classifier.
Model = rho: 0.5, number of selected non-linear variables: 14


/Users/lorrainedalmeida/Work/Experiment_Week/synthsonic/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


Transforming variables.
Fitting and calibrating classifier.
Model = rho: 0.5, number of selected non-linear variables: 14


/Users/lorrainedalmeida/Work/Experiment_Week/synthsonic/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


,adult/accuracy,adult/f1,timestamp
CLBNSynthesizer,0.760117,0.291326,2020-04-12 09:41:35.096775
CTGANSynthesizer,0.784217,0.597420,2020-04-12 09:41:35.096775
IdentitySynthesizer,0.825608,0.661600,2020-04-12 09:41:35.096775
IndependentSynthesizer,0.654425,0.174515,2020-04-12 09:41:35.096775
MedganSynthesizer,0.593683,0.276167,2020-04-12 09:41:35.096775
TableganSynthesizer,0.798850,0.426258,2020-04-12 09:41:35.096775
TVAESynthesizer,0.810817,0.630175,2020-04-12 09:41:35.096775
UniformSynthesizer,0.500242,0.301878,2020-04-12 09:41:35.096775
VEEGANSynthesizer,0.718792,0.162142,2020-04-12 09:41:35.096775
KDECopulaNNPdf_woKDE_FreqMapping,0.750375,0.454029,2020-09-23 12:55:47.158573
